<a href="https://colab.research.google.com/github/renesteeman/BlockchainDiagramITAssignment/blob/master/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install image package
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

# pip installs
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

# Imports
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor
from datasets import Audio
from huggingface_hub import notebook_login

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:1

In [4]:
# Connect to huggingface
# If this doesn't work (I couldn't interact with the widget), use 'huggingface-cli login' instead
# Get the token from Account->Settings->Acess tokens
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [9]:
# Load dataset
common_voice = DatasetDict()

# I like how the sentences don't make much sense, the labels are wrong, and even I can't understand some the voices
# The dataset is too large, so only load part of it (15%)
# Tried 15, 25 with tiny
load_percentage = 25
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "nl", split="train[:"+str(load_percentage)+"%]+validation[:"+str(load_percentage)+"%]", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "nl", split="test[:"+str(load_percentage)+"%]", use_auth_token=True)

# Remove unneeded columns
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10264
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2686
    })
})


In [10]:
# Load model
# To improve performance, one could go for a larger model, but this would require a lot more time and resources to train
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="Dutch", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="Dutch", task="transcribe")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [ ]:
# Pre-process data
# pip install "torchaudio<0.12"
print(common_voice["train"][0])

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# run the prepare_dataset code on the dataset, this takes *a lot* of time
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/006a821ef4381eedd39e2888dec0d506081dd166dee4ff268cdb5df0844b4cba/cv-corpus-11.0-2022-09-21/nl/clips/common_voice_nl_17708448.mp3', 'array': array([ 0.0000000e+00, -3.5979097e-14,  1.1424202e-13, ...,
       -8.1800852e-07,  3.1895601e-07,  1.8009855e-06], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'Tom, Jan en Julian spellen alledrie gitaar in een band.'}
   

#1:   0%|          | 0/5132 [00:00<?, ?ex/s]

#0:   0%|          | 0/5132 [00:00<?, ?ex/s]

In [ ]:
# Time to save the result, so we don't lose it after having waited for 2 hours
# This required more storage than Google allows for, so this costed me $3 (for Google Drive storage)
print(type(common_voice))

# original (tiny-15): common_voice_processed.hf
# tiny-25: common_voice_processed_tiny_25.hf
# base-25: common_voice_processed_base_25.hf
common_voice.save_to_disk("/content/drive/MyDrive/Scalable/lab2/common_voice_processed_base_25.hf")